First we start by importing the libraries we need to use

In [ ]:
from IPython.core.display import display, HTML # We need this to format some output

In [ ]:
import sqlite3

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

We use the following line to format the style of the notebook, to set the width to 100%

In [ ]:
display(HTML("<style>.container { width:100% !important; }; h1 {font-size: 3.5em; line-height:1.4em;text-align:center;} </style>"))

Next we are going to connect the vulnerability Database. (the code to create and populate this DB is in the "Jiras" notebook.

In [ ]:
conn = sqlite3.connect('jiras.db')

In [ ]:
df = pd.read_sql_query("select * from jiras;",conn) # We use Pandas to import the vulns from SQLITE. We can run standard SQL queries via Pandas. 

The data will be imported in a Dataframe. DataFrame is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects

# All issues description

In [ ]:
df.describe()  

# Total vulnerabilities

In [ ]:
display(HTML("<h1>"+str(df['vuln_category'].count())+"</h1>")) ## We use display with HTML format to print out the results in a for

In [ ]:
our_total=df['vuln_category'].count() ## we store the results in our_total variable for later use.

## Vulnerabilities per category

In [ ]:
categories = df[['vuln_category','squad']]

In [ ]:
categories_group=categories.groupby('vuln_category')

In [ ]:
categories_group.describe()

In [ ]:
cat = categories_group.size() ## Compute group sizes

We use this magic function, to embed matplotlib graph inline the Notebook. 

In [ ]:
%matplotlib inline

In [ ]:
myplot = cat.plot(kind='barh') ## We use the plot function from Matplotlib to create a  horizontal bar chart with the data in "cat"

## Open per Squad

In [ ]:
open_squad = df[(df.status != 'Done')].groupby('squad').size() # Same as df_open_squads = pd.read_sql_query("select squad, count(level),level from jiras where status not in ('Done') group by squad,level;",conn)

In [ ]:
a = open_squad.plot(x="squad", y="count" ,kind='barh')

## Open issues per risk level

In [ ]:
open_level = df[(df.status != 'Done')].groupby('level').size()  ## Same as df_open_levels = pd.read_sql_query("select count(level),level from jiras where status not in ('Done') group by level;",conn)

In [ ]:
myplot = open_level.plot.pie(figsize=(6,6),shadow=True,explode=(0.2,0.0,0,0),labels=['Blocker','Critical','Major','Minor'],colors=['crimson','gold','yellow','dodgerblue'],y='count(level)',autopct='%.1f%%S',fontsize=13)

## Open bugs age

We use the Julianday function to get the current date and the date when was created in the same format (days)

In [ ]:
df_open_age = pd.read_sql_query("select julianday('now') - julianday(created) as days, squad from jiras where status not in ('Done');",conn) 

In [ ]:
df_open_age.head(3)

In [ ]:
openavg = df_open_age['days'].mean() ## We use the mean function on the days column of our dataframe to obtain the average time a bug is open.
display(HTML("<h1>"+str(int(openavg))+" days</h1>"))

## Closed bug age

In [ ]:
df_closed_age = pd.read_sql_query("select julianday(closed) - julianday(created) as days, squad from jiras where status in ('Done');",conn)

In [ ]:
df_closed_age.head(2)

In [ ]:
closeavg = df_closed_age['days'].mean()
display(HTML("<h1>"+str(int(closeavg))+" days</h1>"))

## Total Bug age

In [ ]:
ages = [df_closed_age, df_open_age]

In [ ]:
ages_total = pd.concat(ages) ## We concatenate closed age and open age columns to create one table to calculate the total bug age.

In [ ]:
totalavg= ages_total['days'].mean()
display(HTML("<h1>"+str(int(totalavg))+" days</h1>"))


## Reporters HoF

In [ ]:
total_reporters = df.groupby('reporter').size()

In [ ]:
myplot =total_reporters.plot(kind='barh')

In [ ]:
closed_issues = df[(df.status == 'Done')]['id'].count()

## Closed issues

In [ ]:
display(HTML("<h1>"+str(closed_issues)+"</h1>"))

In [ ]:
open_issues = df[(df.status != 'Done')]['id'].count()

## Open issues

In [ ]:
display(HTML("<h1 style='color:red;'>"+str(open_issues)+"</h1>"))

In [ ]:
open_issues_overdue = df[(df.status != 'Done') & (df.overdue == 'True')]['id'].count()

## Overdue issues

In [ ]:
display(HTML("<h1 style='color:red;'>"+str(open_issues_overdue)+"</h1>"))

In [ ]:
source = df[['vuln_source','status','squad']]

In [ ]:
source.head(3)

In [ ]:
source_group=source.groupby('vuln_source')

In [ ]:
cat = source_group.size()

In [ ]:
cat.head()

In [ ]:
mysources = cat.plot(x="source", y="count(level)" ,kind='barh')

## Issues per Tribe

In [ ]:
tribes = df[['vuln_source','status','tribe']]

In [ ]:
tribes_group=tribes.groupby('tribe')

In [ ]:
cat = tribes_group.size()

In [ ]:
myplot = cat.plot(x="tribes", y="count(level)" ,kind='barh')

In [ ]:
overdue_tribe = df[(df.status != 'Done') & (df.overdue == 'True')].groupby('tribe').size()

## Overdue per Tribe

In [ ]:
myplot = overdue_tribe.plot(x="tribe", y="count(*)" ,kind='barh',color='red')

In [ ]:
df_source = pd.read_sql_query("select summary,level from jiras order by created DESC limit 3;",conn)

In [ ]:
df_source

## Whitehat Industry metrics https://info.whitehatsec.com/rs/675-YBI-674/images/WH-2016-Stats-Report-FINAL.pdf

In [ ]:
industry_average= {'technology':295,'retail':456} ## Average time 

In [ ]:
industry_average_TTF= {'technology':118,'retail':205} ## Whitehat Time to fix average per industry

In [ ]:
industry_bugsxsite= {'technology':32,'retail':23} ## Whitehat total issues per site per industry

In [ ]:
if closeavg < industry_average_TTF['technology']: 
    display(HTML("<h1>"+str(int(closeavg))+" days<img src='green.png' height='35' width='35'></h1>"))
else:
    display(HTML("<h1>"+str(int(closeavg))+" days<img src='red.png' height='35' width='35'></h1>"))


In [ ]:
if totalavg < industry_average['technology']: 
    display(HTML("<h1>"+str(int(totalavg))+" days<img src='green.png' height='35' width='35'></h1>"))
else:
    display(HTML("<h1>"+str(int(totalavg))+" days<img src='red.png' height='35' width='35'></h1>"))


In [ ]:
if our_total < industry_bugsxsite['technology']: 
    display(HTML("<h1>"+str(int(our_total))+"<img src='green.png' height='35' width='35'></h1>"))
else:
    display(HTML("<h1>"+str(int(our_total))+"<img src='red.png' height='35' width='35'></h1>"))


In [ ]:
display(HTML("<img src='devseccon.png'>"))  ## We add our cool logo